In [ ]:
import sys
sys.path.append('/media/pato/DATA/Dev-VAM/GPHRM')  
from GPHRM import GPHRM

In [ ]:
from GPHRM import score

In [ ]:
score(3012,3011,1.5,method='MCML',z=1,fitTwice=False)

In [ ]:
import pandas as pd
import yaml
import sqlalchemy
from sqlalchemy import Column, Integer, Float, String, create_engine  
from sqlalchemy.dialects.postgresql import JSON, JSONB

In [ ]:
rsltDF = pd.read_excel('/media/pato/DATA/Dev-VAM/GPHRM/savedModels/prd3012.xlsx')
trainID = 3012
scoreID = 3011
kappa = 0.5
method = 'MCML'
z = 1
mdlFilepath = '/media/pato/DATA/Dev-VAM/GPHRM/savedModels/mdl3012.RData'

#Connect to database
with open('../HRM/private_config.yml', 'r') as cfgfile:
    private_config = yaml.load(cfgfile)

engine = create_engine("""postgresql+psycopg2://{}:{}@{}/{}"""
                   .format(private_config['DB']['user'], private_config['DB']['password'],
                           private_config['DB']['host'], private_config['DB']['database']))

# Prepare SQL
rsltTbl = sqlalchemy.table("gp_predict",Column('train_config_id', Integer),
                                        Column('score_config_id', Integer),
                                        Column('model_save_file', String),
                                        Column('hyperparams',JSON),
                                        Column('predict_gps', JSON),
                                        Column('predict_score', JSON),
                                        Column('predict_error',JSON),
                                        Column('exceednc_prob',JSON))

#compile insert statement
statement = rsltTbl.insert().values(train_config_id = trainID,
                                    score_config_id = scoreID,
                                    model_save_file = mdlFilepath,
                                    hyperparams = {'kappa':kappa,'method':method,'z':z},
                                    predict_gps = rsltDF.iloc[:,[0,1,2,3]].to_json(),
                                    predict_score = rsltDF.iloc[:,[0,1,4]].to_json(),
                                    predict_error = rsltDF.iloc[:,[0,1,5,6,7,8,9,10]].to_json(),
                                    exceednc_prob = rsltDF.iloc[:,[0,1,11,12,13,14,15]].to_json()
                                   )

#Execute insert
engine.execute(statement)
print('finished writing results to database - creating raster')

In [1]:
import pandas as pd
rsltDF = pd.read_excel('/media/pato/DATA/Dev-VAM/GPHRM/savedModels/prd3012.xlsx')

In [2]:
import numpy as np
import pdb

In [18]:
rsltDF['Category'] = rsltDF.loc[:,['x5%','x10%','x20%','x35%','x50%']].apply(lambda x: np.argmin([100 if i<0.85 else i for i in x.values])+2,axis=1)
rsltDF.loc[rsltDF['x5%']<=0.85,'Category'] = 1

In [4]:
import os
import sys
sys.path.append('/media/pato/DATA/Dev-VAM/HRM')  

In [5]:
from Src import utils as HRMutils

In [10]:
rsltDF.columns

Index(['i', 'j', 'gpsLatitude', 'gpsLongitude', 'prevalence', 'stdError', '5%',
       '10%', '50%', '90%', '95%', 'x5%', 'x10%', 'x20%', 'x35%', 'x50%',
       'Category'],
      dtype='object')

In [11]:
HRMutils.tifgenerator('/media/pato/DATA/Dev-VAM/GPHRM/SierraLeoneStdError.tif',
                      '/media/pato/DATA/Dev-VAM/GPHRM/SierraLeonePop/SLE14adjv1.tif',
                      rsltDF,'stdError',aggregate_factor=10,crop=True)

INFO: aggregating raster ...
3701
-> writing:  /media/pato/DATA/Dev-VAM/GPHRM/SierraLeoneStdError.tif


In [12]:
from shapely.ops import polygonize
from shapely.geometry import LineString, MultiPolygon, MultiPoint, Point
from scipy.spatial import Voronoi

points = rsltDF.loc[:,['gpsLatitude','gpsLongitude']].values

vor = Voronoi(points)

lines = [LineString(vor.vertices[line])for line in vor.ridge_vertices if -1 not in line]

convex_hull = MultiPoint([Point(i) for i in points]).convex_hull.buffer(2)
result = MultiPolygon([poly.intersection(convex_hull) for poly in polygonize(lines)])

#pts = MultiPoint([Point(i) for i in points])
#mask = pts.convex_hull.union(pts.buffer(10, resolution=5, cap_style=3))
#result = MultiPolygon([poly.intersection(mask) for poly in polygonize(lines)])

In [24]:
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from scipy.spatial import voronoi_plot_2d

%matplotlib qt
    
# normalize chosen colormap
norm = mpl.colors.Normalize(vmin=1, vmax=6, clip=True)
mapper = cm.ScalarMappable(norm=norm, cmap=cm.RdYlGn)

# plot Voronoi diagram, and fill finite regions with color mapped from speed value
voronoi_plot_2d(vor, show_points=False, show_vertices=False, s=1)
for r in range(len(vor.point_region)):
    region = vor.regions[vor.point_region[r]]
    if not -1 in region:
        polygon = [vor.vertices[i] for i in region]
        plt.fill(*zip(*polygon), color=mapper.to_rgba(7-rsltDF.loc[r,'Category']))
plt.show()

In [19]:
rsltDF['Category'].unique()

array([5, 6, 4, 2, 3, 1])